In [1]:
# Data manipulation
import pandas as pd

# Visualization
import plotly.express as px

# Sklearn
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.svm import SVC # for Support Vector Classification baseline model
from sklearn.semi_supervised import SelfTrainingClassifier # for Semi-Supervised learning
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.metrics import f1_score

from sklearn.utils import shuffle
import math
import os
import numpy as np

from keras.layers import Input, Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import keras.backend as K

import time
import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Read in data
FOLDER = "/content/drive/MyDrive/Emotion-RST/marketing_campaign.csv"
df = pd.read_csv(FOLDER, 
                 encoding='utf-8', delimiter=';',
                 usecols=['ID', 'Year_Birth', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'MntWines', 'MntMeatProducts'])
# Create a flag to denote whether the person has any dependants at home (either kids or teens)
df['Dependents_Flag']=df.apply(lambda x: 1 if x['Kidhome']+x['Teenhome']>0 else 0, axis=1)

# Print datafram

In [3]:
#将训练集划分为子训练集、验证集，比例是0.75:0.25
df_train, df_test = train_test_split(df, test_size=0.25, random_state=0)
print('Size of train dataframe: ', df_train.shape[0])
print('Size of test dataframe: ', df_test.shape[0])
print("#"*50)

#现在让我们屏蔽训练数据中 95% 的标签，并创建一个使用“-1”表示未标记（屏蔽）数据的目标变量：
# Create a flag for label masking
df_train['Random_Mask'] = True
df_train.loc[df_train.sample(frac=0.05, random_state=0).index, 'Random_Mask'] = False
# Create a new target colum with labels. The 1's and 0's are original labels and -1 represents unlabeled (masked) data
df_train['Dependents_Target']=df_train.apply(lambda x: x['Dependents_Flag'] if x['Random_Mask']==False else -1, axis=1)
# Show target value distribution
print('Target Value Distribution:')
print(df_train['Dependents_Target'].value_counts())

#创建Ds,Du,Ds_dev
Ds = df_train[df_train['Dependents_Target']!=-1] #labeled training data in masked Dataset
Du = df_train[df_train['Dependents_Target']==-1] #unlabeled training data in masked Dataset
Ds_dev = df_test

print("#"*50)
print('Size of labeled seen data Ds dataframe: ', Ds.shape[0])
print('Size of unlabeled unseen data Du dataframe: ', Du.shape[0])
print('Size of seen validation set Ds_dev dataframe: ', Ds_dev.shape[0])

Size of train dataframe:  1680
Size of test dataframe:  560
##################################################
Target Value Distribution:
-1    1596
 1      58
 0      26
Name: Dependents_Target, dtype: int64
##################################################
Size of labeled seen data Ds dataframe:  84
Size of unlabeled unseen data Du dataframe:  1596
Size of seen validation set Ds_dev dataframe:  560


In [4]:
def build_model():
    """basic model.
    """
    inputs = Input(shape=(4,), name='ob_input')
    x = Dense(16, activation='relu')(inputs)
    x = Dense(16, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=x)
    return model

def calloss(y_true, y_pred):
      """loss function.
      Arguments:
          y_true: (action, reward)
          y_pred: action_prob

      Returns:
          loss: reward loss
      """
      action_pred = y_pred
      action_true, discount_episode_reward = y_true[:, 0], y_true[:, 1]

      action_true = K.reshape(action_true, (-1, 1))
      loss = K.binary_crossentropy(action_true, action_pred)
      loss = loss * K.flatten(discount_episode_reward)

      return loss

gamma = 0.75    
def discount_reward(rewards):
      """Discount reward
      Arguments:
          rewards: rewards in a episode.
      """
      # compute the discounted reward backwards through time.
      discount_rewards = np.zeros_like(rewards, dtype=np.float32)
      cumulative = 0.
      for i in reversed(range(len(rewards))):
          cumulative = cumulative * gamma + rewards[i]
          discount_rewards[i] = cumulative

      # size the rewards to be unit normal (helps control the gradient estimator variance).
      discount_rewards -= np.mean(discount_rewards)
      discount_rewards //= np.std(discount_rewards)

      discount_rewards = discount_rewards+1
      return list(discount_rewards)

def getnewBpk(Bk,probs):
  Bpk = Bk.copy()
  actions = []
  for i in range(len(probs)):
    prob = probs[i][0]
    action = np.random.choice(np.array(range(2)), size=1, p=[1 - prob, prob])[0]
    actions.append(action)
  Bpk['action'] = actions
  Bpk = Bpk[Bpk['action']!=0]
  return Bpk,actions

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [55]:
svc_model = SVC(kernel='rbf', 
    probability=True, 
    C=1.0, # default = 1.0
    gamma='scale', # default = 'scale'
    random_state=0
)

policy_model = build_model()
policy_model.compile(loss=calloss, optimizer=Adam(lr=0.01))

class DataSelectController:
    def __init__(self,df_train,df_test):
        self.df_train = df_train #
        self.df_test = df_test #

        self.labeled_train_dataset = df_train[df_train['Dependents_Target']!=-1] #
        self.unlabeled_train_dataset = df_train[df_train['Dependents_Target']==-1] #unlabeled training data in masked Dataset
        
        self.validation_dataset = df_test#
        self.unlabeled_validation_dataset = pd.DataFrame(columns=df_test.columns)

        self.pseudo_labeled_dataset = pd.DataFrame(columns=self.labeled_train_dataset.columns)
        #self.pseudo_validation_dataset = pd.DataFrame(columns=self.labeled_train_dataset.columns)

        self.base_svc_model = self.train_svc_model()
        self.policy_model = build_model()
        self.policy_model.compile(loss=calloss, optimizer=Adam(lr=0.01))
        pass

    #inital environment
    def reset(self):
        self.labeled_train_dataset = self.df_train[df_train['Dependents_Target']!=-1] #
        self.unlabeled_train_dataset = self.df_train[df_train['Dependents_Target']==-1] #unlabeled training data in masked Dataset
        
        self.validation_dataset = self.df_test
        self.unlabeled_validation_dataset = pd.DataFrame(columns=self.df_test.columns)

        self.pseudo_labeled_dataset = pd.DataFrame(columns=self.labeled_train_dataset.columns)
        #self.pseudo_validation_dataset = pd.DataFrame(columns=self.labeled_train_dataset.columns)
        self.base_svc_model = self.train_svc_model()
        pass

    #update environment,每一个episode，更新一次pseudo_labeled_dataset，pseudo_validation_dataset，unlabeled_train_dataset
    def render(self):                     
        pass

    #Train svc_model using trainset
    def train_svc_model_xy(self,trainset_x,trainset_y):
        clf = svc_model.fit(trainset_x,trainset_y)
        self.base_svc_model = clf
        return clf
    
    def train_svc_model(self):
        D_train_ = self.labeled_train_dataset if self.pseudo_labeled_dataset.empty else pd.concat([self.labeled_train_dataset, self.pseudo_labeled_dataset])
        clf = svc_model.fit(D_train_[['MntMeatProducts', 'MntWines']], D_train_['Dependents_Flag'].values)
        self.base_svc_model = clf
        return clf

    def predict_unlabeled_set(self, unlabeled_set):
        proba = self.base_svc_model.predict_proba(unlabeled_set[['MntMeatProducts', 'MntWines']])
        predicted_label = self.base_svc_model.predict(unlabeled_set[['MntMeatProducts', 'MntWines']])

        U = unlabeled_set.copy()
        U['proba_A'] = [a[0] for a in proba]
        U['proba_B'] = [b[1] for b in proba]
        U['predicted_label'] = predicted_label

        U_predicted = U
        return U_predicted

    def rank_U(self,U_predicted):
        U_predicted['proba_max'] = U_predicted.loc[:,['proba_A','proba_B']].T.max()
        U_ranked = U_predicted.sort_values('proba_max',ascending = False)
        U_ranked = U_ranked.drop(columns=['proba_max'])
        return U_ranked


    def select_Batch_k_PolicyNetwrok(self,policy_model,Batch_k):
        Batch_k_judged = Batch_k.copy()
        data_ = Batch_k[['MntMeatProducts','MntWines','proba_A','proba_B']]
        data_ = (data_-data_.min())/(data_.max()-data_.min())
        confidences = policy_model.predict(data_)
        print(confidences.T)
        #input()
        actions = []
        for i in range(len(confidences)):
            prob = confidences[i][0]
            #if(prob==None):
            action = np.random.choice(np.array(range(2)), size=1, p=[1 - prob, prob])[0]
            actions.append(action)
        Batch_k_judged['action'] = actions
        Batch_k_selected = Batch_k_judged.copy()
        Batch_k_selected = Batch_k_selected[Batch_k_selected['action']!=0]
        Batch_k_unselected = Batch_k_judged.copy()
        Batch_k_unselected = Batch_k_unselected[Batch_k_unselected['action']==0]
        return Batch_k_judged, Batch_k_selected, Batch_k_unselected, actions

    def cal_f1(self,clf_,validation_set):

        y_pred = clf_.predict(validation_set[['MntMeatProducts', 'MntWines']])
        #validation_score_list.append(clf_.score(validation_dataset[['MntMeatProducts', 'MntWines']], validation_dataset['Dependents_Flag'].values))
        validation_set_f1 = f1_score(validation_set['Dependents_Flag'].values, y_pred, average='macro')##TODO

        return validation_set_f1
        pass

    #对当前状态的Unlabeled_set按照Batch分组进行Policy Network评估，再合并计算各自的Reward
    def step(self,U, batch):
        observation = None
        reward=0
        done = False
        info = ""

        U = shuffle(U)#打乱U
        every_epoch_num = math.floor((len(U)/batch))

        Batch_k_list = []
        Batch_actions_list=[]
        Batch_k_judged_list = []
        validation_f1_list= []
        pseudo_f1_list = []
        validation_score_list=[]
        pseudo_score_list=[]
        for k in range(batch):
            #get a batch Bk from U
            if k < (batch-1):
                Batch_k = U[every_epoch_num * k: every_epoch_num * (k + 1)]
            else:
                Batch_k = U[every_epoch_num * k:]
            
            Batch_k_judged, Batch_k_selected, Batch_k_unselected, actions = self.select_Batch_k_PolicyNetwrok(policy_model, Batch_k)
            Batch_k_list.append(Batch_k)
            Batch_actions_list.append(actions)
            Batch_k_judged_list.append(Batch_k_judged)
      
            #Train model clf' with Bpk
            try:
              clf_ = svc_model.fit(Batch_k_selected[['MntMeatProducts', 'MntWines']], Batch_k_selected['predicted_label'].values)
            except ValueError as e:
              print ('error type: ', type (e))
              print(Batch_k_selected['predicted_label'].values)
              done = True
              return observation, reward, done, info

            validation_f1 = self.cal_f1(clf_,self.validation_dataset)
            validation_score_list.append(clf_.score(self.validation_dataset[['MntMeatProducts', 'MntWines']], self.validation_dataset['Dependents_Flag'].values))
            validation_f1_list.append(validation_f1)

            if(self.pseudo_labeled_dataset.empty!=True):
                pseudo_f1 = self.cal_f1(clf_,self.pseudo_labeled_dataset)
                pseudo_f1_list.append(pseudo_f1)
                pseudo_score_list.append(clf_.score(self.pseudo_labeled_dataset[['MntMeatProducts', 'MntWines']], self.pseudo_labeled_dataset['Dependents_Flag'].values))

        validation_f1_mean = np.mean(validation_f1_list)
        validation_f1_std = np.std(validation_f1_list)

        pseudo_f1_mean = np.mean(pseudo_f1_list)
        pseudo_f1_std = np.std(pseudo_f1_list)
         
        for k in range(batch):
            reward1 = (validation_f1_list[k] - validation_f1_mean)/validation_f1_std
            reward2 = 0
            lamda =0
            if(self.pseudo_labeled_dataset.empty!=True):
              reward2 = (pseudo_f1_list[k] - pseudo_f1_mean)/pseudo_f1_std
              lamda =len(self.validation_dataset)/len(self.pseudo_labeled_dataset)

            #reward = reward1 + lamda*reward2
            reward = reward1
            
            Batch_k = Batch_k_list[k]
            Batch_actions = Batch_actions_list[k]
            Batch_rewards = [reward]*len(Batch_actions)
            X = Batch_k[['MntMeatProducts', 'MntWines','proba_A','proba_B']]
            X = (X-X.min())/(X.max()-X.min())
            #print("Batch_actions")
            print("  ",Batch_actions)
            #print(Batch_k['Dependents_Flag'].values)
            y = np.array(list(zip(Batch_actions, Batch_rewards)))

            loss = policy_model.train_on_batch(X, y)

            validation_score =validation_score_list[k]
            pseudo_score=0
            if(self.pseudo_labeled_dataset.empty!=True):
              pseudo_score = pseudo_score_list[k]
            print('   Batch: {} |Batch reward: {} |loss: {:.3f} |validation_score: {:.3f} |clf_pseudo_score: {:.3f}'.format( k,reward,loss,validation_score,pseudo_score))

        #完成后，需要对各个临时数据存储进行更新
          #集合全部Batch_k_judged，根据'action'字段选出selected和unselected
          #将selected添加到self.pseudo_labeled_dataset
          #将selected从self.unlabeled_train_dataset中去除
        U_judged = pd.concat(Batch_k_judged_list)#集合全部Batch_k_judged，
        U_selected = U_judged[U_judged['action']==1]#根据'action'字段选出selected和unselected
        U_unselected = U_judged[U_judged['action']!=1]#根据'action'字段选出selected和unselected


        U_selected = U_selected.drop(['proba_A', 'proba_B','action'], axis=1)
        self.pseudo_labeled_dataset = U_selected if self.pseudo_labeled_dataset.empty else pd.concat([self.pseudo_labeled_dataset, U_selected])#dataframe 参数不同，有bug，TODO        
        
        U_selected = U_selected.drop(['predicted_label'], axis=1)
        self.unlabeled_train_dataset = pd.concat([self.unlabeled_train_dataset, U_selected, U_selected]).drop_duplicates(keep=False)
        
        
        #print(self.unlabeled_train_dataset.columns)
        #print(U_selected.columns)
        print('U_selected:{}| pseudo_labeled_dataset:{}| unlabeled_train_dataset:{}|'.format(len(U_selected),len(self.pseudo_labeled_dataset),len(self.unlabeled_train_dataset)))
        #input()

        return observation, reward, done, info


iteration = 1000
episode = 10 #5000
batch = 7
Controller = DataSelectController(df_train, df_test)    
done=False

for i in range(iteration):
    Controller.reset()#恢复数据集，初始化临时数据存储
    
    for j in range(episode):
        print("iteration",i,"Episode:",j)

        Controller.train_svc_model()#Step1：训练svc class matching model（合并labeled dataset和pseudo_labeled_dataset）
        U_predicted = Controller.predict_unlabeled_set(Controller.unlabeled_train_dataset)
        observation, reward, done, info = Controller.step(U_predicted, batch) #
        Controller.render()#更新临时数据存储
        if(done==True):
          done=False
          break


Output hidden; open in https://colab.research.google.com to view.